In [1]:
import rospy
import numpy as np
from turtlesim.srv import Spawn
from turtlesim.srv import Kill
from turtlesim.srv import SetPen
from turtlesim.msg import Pose
from geometry_msgs.msg import Twist
from pkg_tsr.srv import Task
from std_msgs.msg import Float32
import math

rospy.init_node("Robot")

In [2]:
kill_robot = rospy.ServiceProxy('kill',Kill)
class Robot():
    position = Pose()
    def __init__(self, name, x, y):
        self.name = name
        create_new_turtle = rospy.ServiceProxy('spawn', Spawn)
        create_new_turtle(x, y, 0, name)
        
        position = Pose(x,y,0,0,0)
        self.batteryDrain = rospy.get_param("robot/batteryDrain")
        self.batteryCharge = rospy.get_param("robot/batteryCharge")
        
        self.set_pen = rospy.ServiceProxy(name+'/set_pen',SetPen)
        self.set_pen(0,0,0,0,1)
        
        self.follow_service = rospy.Service(name+'/go_to_goal', Task, self.go_to_goal)
        self.velocity_pub = rospy.Publisher(name+'/cmd_vel', Twist, queue_size=10)
        self.pose_sub = rospy.Subscriber(name+'/pose',Pose, self.callback_pose)
        self.battery_pub = rospy.Publisher(name+'/battery',Float32, queue_size=10)
        self.battery = 100.0
        
        rospy.set_param(self.name+"/tryb_pracy","autonomiczny")
        
    def go_to_goal(self, msg_srv):
        max_speed = 0.3 # dla liniowych i obrotowych
        tolerance = 0.05 # max odleglosc od punktu
        
        # obracanie się w stronę punktu
        while True:
            celTheta = math.atan2(msg_srv.y - self.position.y,msg_srv.x - self.position.x)
            celDelta = celTheta - self.position.theta
            if abs(celDelta) < tolerance*0.005:
                break
            # poprawiamy
            msg_vel = Twist()
            speed = np.clip(0.2*(abs(celDelta))+0.01,0,max_speed)
            if celDelta > 0:
                msg_vel.angular.z = speed
            else:
                msg_vel.angular.z = -speed
            if rospy.get_param(self.name+'/tryb_pracy')=='autonomiczny':
                self.velocity_pub.publish(msg_vel)
            else:
                msg_vel.angular.z=0
                self.velocity_pub.publish(msg_vel)
                return "Interrupted"
            
        # jazda do punktu
        while True:
            celDelta = math.sqrt((msg_srv.x - self.position.x)**2 + (msg_srv.y - self.position.y)**2)
            if celDelta < tolerance:
                break
            #poprawiamy
            msg_vel = Twist()
            msg_vel.linear.x = np.clip(0.25*celDelta+0.025,0,max_speed)
            if rospy.get_param(self.name+'/tryb_pracy')=='autonomiczny':
                self.velocity_pub.publish(msg_vel)
            else:
                msg_vel.linear.x=0
                self.velocity_pub.publish(msg_vel)
                return "Interrupted"
            
        return "finished" 
        
    def callback_pose(self, data):
        #deltaX = data.x - self.position.x
        #deltaY = data.y - self.position.y
        #distance = math.sqrt( deltaX**2 + deltaY**2)
        if (data.x >= 9.5 and data.y >= 9.5) or (data.x <= 1.5 and data.y <= 1.5):
            self.battery += self.batteryCharge
        else:
            self.battery -= data.linear_velocity * self.batteryDrain
        self.battery = np.clip(self.battery,0,100)
        self.battery_pub.publish(self.battery)
        self.position = data
    
    def get_params(self):
        try:
            return rospy.get_param(self.name)
        except KeyError:
            print("Brak parametrow")
            
    def unregister(self):
        self.follow_service.shutdown()
        self.pose_sub.unregister()
        kill_robot(self.name)
        
# funkcja pomocnicza do kierowania robotem
def kieruj(robot,listaPunktow):
    klient = rospy.ServiceProxy(robot.name+'/go_to_goal',Task)
    for punkt in listaPunktow:
        klient(punkt.x,punkt.y)
        
class Punkt():
    x = 0.0
    y = 0.0
    def __init__(self,x,y):
        self.x = x
        self.y = y

In [3]:
!rosservice call reset

In [7]:
robocik.unregister()

In [3]:
# UZUPEŁNIĆ - utworzyć klienta i zlecić zadanie dojazdu do kilku różnych punktów
robocik = Robot("Danonek4",4,3)
droga = [ 
    Punkt(5,5),
    Punkt(5,8)
    #Punkt(8,8),
    #Punkt(1,5),
    #Punkt(5,4)
]
#kieruj(robocik,droga)

In [4]:
robocik.get_params()

{'tryb_pracy': 'autonomiczny'}

In [5]:
robocik.position

x: 4.0
y: 3.0
theta: 0.0
linear_velocity: 0.0
angular_velocity: 0.0

In [4]:
roboty = [
    Robot("Danonek4",4,3),
    Robot("Zenonek",8,3)
]

[WARN] [1625573045.345765]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
[WARN] [1625573158.085335]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
[WARN] [1625573992.416643]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.


In [ ]:
for robot in roboty:
    robot.unregister()